In [2]:
!pip install transformers

In [4]:
!pip install PyMuPDF

   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
    --------------------------------------- 0.3/16.6 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.6 MB 2.4 MB/s eta 0:00:07
   --- ------------------------------------ 1.3/16.6 MB 2.5 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/16.6 MB 2.8 MB/s eta 0:00:06
   ------ --------------------------------- 2.9/16.6 MB 2.9 MB/s eta 0:00:05
   -------- ------------------------------- 3.7/16.6 MB 3.1 MB/s eta 0:00:05
   ----------- ---------------------------- 4.7/16.6 MB 3.4 MB/s eta 0:00:04
   ------------- -------------------------- 5.8/16.6 MB 3.5 MB/s eta 0:00:04
   --------------- ------------------------ 6.6/16.6 MB 3.6 MB/s eta 0:00:03
   ------------------ --------------------- 7.6/16.6 MB 3.7 MB/s eta 0:00:03
   -------------------- ------------------- 8.4/16.6 MB 3.7 MB/s eta 0:00:03
   ---------------------- ----------------- 9.2/16.6 MB 3.7 MB/s eta 0:00:03
   ----------

In [1]:
!pip install nltk

In [1]:


---

### **1. Cài đặt thư viện cần thiết**
Trước tiên, hãy cài đặt các thư viện cần dùng:
```bash
pip install pandas pymupdf pdfminer.six transformers vncorenlp underthesea torch
```

---

### **2. Đọc danh sách từ khóa từ file Excel**
Chúng ta sử dụng `pandas` để đọc danh sách từ khóa từ file Excel:
```python
import pandas as pd

# Đọc file Excel chứa danh sách từ khóa
file_excel = "keywords.xlsx"
df_keywords = pd.read_excel(file_excel)
keywords = df_keywords['Keyword'].tolist()  # Lấy danh sách từ khóa
```

---

### **3. Trích xuất văn bản từ file PDF**
Ta có thể dùng `PyMuPDF` hoặc `pdfminer.six` để lấy văn bản từ file PDF:
```python
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Đọc nội dung từ file PDF
pdf_file = "company_report.pdf"
document_text = extract_text_from_pdf(pdf_file)
```

---

### **4. Tiền xử lý văn bản**
Ta làm sạch văn bản để chuẩn bị cho phân tích:
```python
import re
import underthesea  # Dùng để tách từ tiếng Việt

def preprocess_text(text):
    text = text.lower()  # Chuyển về chữ thường
    text = re.sub(r'\s+', ' ', text)  # Xóa khoảng trắng dư thừa
    words = underthesea.word_tokenize(text)  # Tách từ tiếng Việt
    return words

processed_text = preprocess_text(document_text)
```

---

### **5. Tìm kiếm từ khóa và gán nhãn**
Chúng ta kiểm tra xem từ khóa có xuất hiện trong văn bản hay không. Nếu có, gán nhãn `1`, nếu không gán `0`:
```python
def check_keywords(text, keywords):
    found_keywords = []
    for keyword in keywords:
        if keyword in text:
            found_keywords.append(keyword)
    return 1 if found_keywords else 0  # Gán 1 nếu tìm thấy từ khóa

label = check_keywords(processed_text, keywords)
print(f"Kết quả phân tích: {label}")
```

---

### **6. Mở rộng tìm kiếm với PhoBERT để nhận diện ngữ cảnh**
Nếu muốn mở rộng khả năng nhận diện ngữ cảnh của từ khóa, ta sử dụng **PhoBERT**, một mô hình NLP mạnh mẽ dành cho tiếng Việt:
```python
from transformers import AutoModel, AutoTokenizer

# Khởi tạo PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def analyze_context(text, keywords):
    tokens = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    output = model(**tokens)

# Kiểm tra sự xuất hiện của từ khóa trong các embeddings (giản lược)
    for keyword in keywords:
        if keyword in text:
            return 1
    return 0

label_context = analyze_context(document_text, keywords)
print(f"Kết quả phân tích ngữ cảnh: {label_context}")
```

---

### **7. Xuất kết quả ra file**
Cuối cùng, ta có thể xuất kết quả ra file CSV để dễ dàng theo dõi:
```python
result_df = pd.DataFrame({'File': [pdf_file], 'Label': [label], 'Context_Label': [label_context]})
result_df.to_csv("output.csv", index=False)
```

---

### **Tóm tắt**
- **Đọc danh sách từ khóa** từ Excel.
- **Trích xuất văn bản** từ PDF.
- **Tiền xử lý** để chuẩn hóa dữ liệu.
- **Tìm kiếm từ khóa trực tiếp** trong văn bản.
- **Mở rộng tìm kiếm** bằng PhoBERT để nhận diện ngữ cảnh.
- **Xuất kết quả** ra file CSV.

Nếu bạn muốn cải thiện mô hình, có thể áp dụng kỹ thuật **embedding nâng cao** hoặc **fine-tuning PhoBERT** để tăng độ chính xác.

SyntaxError: invalid syntax (117876230.py, line 1)

- Chỉnh cosine giữa 0.6 và 0.7
- Chỉnh cách ngắt câu + cách parse file pdf
- 

In [3]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    
    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        try:
            doc = fitz.open(pdf_path)
            full_text = []
            
            for page in doc:
                text = page.get_text("text")
                processed_text = self.preprocess_text(text)
                full_text.append(processed_text)
            
            doc.close()
            return ' '.join(full_text)
        except Exception as e:
            print(f"Error processing PDF {pdf_path}: {e}")
            return ""
    
    def get_embeddings(self, texts, batch_size=32):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            processed_batch = [self.preprocess_text(text) for text in batch]
            
            encoded = self.pho_tokenizer(processed_batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []
    
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract and preprocess text
        pdf_text = self.extract_text_from_pdf(pdf_path)
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into chunks
        chunk_size = 1000
        chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
        if not chunks:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Get embeddings
        try:
            criteria_embeddings = self.get_embeddings(criteria_phrases)
            chunk_embeddings = self.get_embeddings(chunks)
            
            # Ensure proper shapes
            if len(criteria_embeddings.shape) == 1:
                criteria_embeddings = criteria_embeddings.reshape(1, -1)
            if len(chunk_embeddings.shape) == 1:
                chunk_embeddings = chunk_embeddings.reshape(1, -1)
            
            # Calculate similarities
            similarities = cosine_similarity(criteria_embeddings, chunk_embeddings)
            
            # Initialize and update results
            results = {}
            for i, phrase in enumerate(criteria_phrases):
                results[phrase] = 1 if np.max(similarities[i]) >= self.similarity_threshold else 0
            
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}
    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")

# Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
    analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing TH_milk.pdf...
Processing VN_milk.pdf...
Results saved to esg_analysis_results.xlsx


In [1]:
!pip install pdfplumber pytesseract

In [4]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Enhanced PDF text extraction"""
        try:
            # Method 1: Regular text extraction
            doc = fitz.open(pdf_path)
            regular_text = []
            for page in doc:
                text = page.get_text("text")
                regular_text.append(text)
            doc.close()
            
            # Method 2: Layout-aware extraction
            import pdfplumber
            with pdfplumber.open(pdf_path) as pdf:
                layout_text = []
                for page in pdf.pages:
                    text = page.extract_text(layout=True)
                    layout_text.append(text)
            
            # Combine both methods
            combined_text = []
            for reg, lay in zip(regular_text, layout_text):
                # Use layout text if it's longer (might contain more information)
                combined_text.append(lay if len(lay) > len(reg) else reg)
            
            # Process the combined text
            processed_text = []
            for text in combined_text:
                processed = self.preprocess_text(text)
                processed_text.append(processed)
            
            return ' '.join(processed_text)
        except Exception as e:
            print(f"Error processing PDF {pdf_path}: {e}")
            return ""
   
    def get_embeddings(self, texts, batch_size=32):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            processed_batch = [self.preprocess_text(text) for text in batch]
            
            encoded = self.pho_tokenizer(processed_batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []
    
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract and preprocess text
        pdf_text = self.extract_text_from_pdf(pdf_path)
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into chunks
        chunk_size = 1000
        chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
        if not chunks:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Get embeddings
        try:
            criteria_embeddings = self.get_embeddings(criteria_phrases)
            chunk_embeddings = self.get_embeddings(chunks)
            
            # Ensure proper shapes
            if len(criteria_embeddings.shape) == 1:
                criteria_embeddings = criteria_embeddings.reshape(1, -1)
            if len(chunk_embeddings.shape) == 1:
                chunk_embeddings = chunk_embeddings.reshape(1, -1)
            
            # Calculate similarities
            similarities = cosine_similarity(criteria_embeddings, chunk_embeddings)
            
            # Initialize and update results
            results = {}
            for i, phrase in enumerate(criteria_phrases):
                results[phrase] = 1 if np.max(similarities[i]) >= self.similarity_threshold else 0
            
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}
    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")

# Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
    analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing TH_milk.pdf...
Processing VN_milk.pdf...
Results saved to esg_analysis_results.xlsx


In [ ]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import nltk
from typing import List, Dict, Tuple
import logging
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")
        
        # Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()
        self.token = Tokenizer.Tokenizer()
        
        # Set up logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold
        self.chunk_overlap = 200  # Number of characters to overlap between chunks
        self.max_chunk_size = 1000  # Maximum size of each chunk
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def segment_words(self, text: str) -> str:
        """Segment Vietnamese text using vws"""
        try:
            segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
            return segmented_text
        except Exception as e:
            self.logger.error(f"Error in word segmentation: {e}")
            return text
    
    def clean_text(self, text: str) -> str:
        """Clean and normalize Vietnamese text for ESG content"""
        try:
            # Convert to lowercase
            text = text.lower()
            
            # Remove special characters but keep Vietnamese characters and numbers
            text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
            
            # Remove extra whitespace
            text = re.sub(r'\s+', ' ', text)
            text = text.strip()
            
            return text
        except Exception as e:
            self.logger.error(f"Error in text cleaning: {e}")
            return text
    
    def remove_stopwords(self, text: str) -> str:
        """Remove Vietnamese stopwords"""
        try:
            words = text.split()
            filtered_words = [word for word in words if word not in self.stopwords]
            return ' '.join(filtered_words)
        except Exception as e:
            self.logger.error(f"Error in stopword removal: {e}")
            return text
    
    def preprocess_text(self, text: str) -> str:
        """Complete text preprocessing pipeline"""
        try:
            # Clean text
            text = self.clean_text(text)
            
            # Remove stopwords
            text = self.remove_stopwords(text)
            
            # Use vws for word segmentation
            segmented_text = self.segment_words(text)
            
            return segmented_text
        except Exception as e:
            self.logger.error(f"Error in text preprocessing: {e}")
            return text

    def extract_text_from_pdf(self, pdf_path: str) -> List[str]:
        """Enhanced PDF text extraction with structure preservation"""
        try:
            doc = fitz.open(pdf_path)
            pages_content = []
            
            for page_num, page in enumerate(doc):
                # Extract text with layout preservation
                blocks = page.get_text("dict")["blocks"]
                page_text = []
                
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            if "spans" in line:
                                line_text = " ".join(span["text"] for span in line["spans"])
                                if line_text.strip():
                                    page_text.append(line_text)
                
                # Join lines with proper spacing
                page_content = " ".join(page_text)
                if page_content.strip():
                    pages_content.append(page_content)
            
            doc.close()
            return pages_content
            
        except Exception as e:
            self.logger.error(f"Error processing PDF {pdf_path}: {e}")
            return []

    def create_smart_chunks(self, text: str) -> List[str]:
        """Create chunks based on natural boundaries"""
        try:
            chunks = []
            sentences = sent_tokenize(text)
            current_chunk = []
            current_size = 0
            
            for sentence in sentences:
                sentence_size = len(sentence)
                
                if current_size + sentence_size > self.max_chunk_size and current_chunk:
                    # Join current chunk and add to chunks
                    chunks.append(" ".join(current_chunk))
                    # Keep last few sentences for overlap
                    overlap_sentences = current_chunk[-2:] if len(current_chunk) > 2 else current_chunk
                    current_chunk = overlap_sentences
                    current_size = sum(len(s) for s in overlap_sentences)
                
                current_chunk.append(sentence)
                current_size += sentence_size
            
            if current_chunk:
                chunks.append(" ".join(current_chunk))
            
            return chunks
        except Exception as e:
            self.logger.error(f"Error in chunk creation: {e}")
            return [text]

    def get_embeddings(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Enhanced embedding generation with normalization"""
        try:
            if isinstance(texts, str):
                texts = [texts]
            
            all_embeddings = []
            
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i + batch_size]
                processed_batch = [self.preprocess_text(text) for text in batch]
                
                encoded = self.pho_tokenizer(processed_batch, 
                                           padding=True, 
                                           truncation=True,
                                           return_tensors="pt",
                                           max_length=256)
                
                # Move inputs to device
                encoded = {k: v.to(self.device) for k, v in encoded.items()}
                
                with torch.no_grad():
                    outputs = self.model(**encoded)
                
                # Get CLS token embeddings and normalize them
                batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
                # L2 normalization
                batch_embeddings = batch_embeddings / np.linalg.norm(batch_embeddings, axis=1, keepdims=True)
                all_embeddings.append(batch_embeddings)
            
            return np.vstack(all_embeddings)
        except Exception as e:
            self.logger.error(f"Error in embedding generation: {e}")
            return np.array([])

    def calculate_similarity(self, vec1: np.ndarray, vec2: np.ndarray) -> float:
        """Calculate similarity using multiple metrics"""
        try:
            # Cosine similarity
            cos_sim = cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))[0][0]
            
            # L2 distance (normalized)
            l2_dist = np.linalg.norm(vec1 - vec2)
            l2_sim = 1 / (1 + l2_dist)
            
            # Combine metrics (weighted average)
            combined_sim = 0.7 * cos_sim + 0.3 * l2_sim
            return combined_sim
        except Exception as e:
            self.logger.error(f"Error in similarity calculation: {e}")
            return 0.0

    def analyze_single_pdf(self, pdf_path: str, criteria_phrases: List[str]) -> Dict[str, int]:
        """Enhanced PDF analysis with improved chunking and similarity calculation"""
        try:
            # Extract text with structure preservation
            pages_content = self.extract_text_from_pdf(pdf_path)
            if not pages_content:
                return {phrase: 0 for phrase in criteria_phrases}
            
            # Create smart chunks for each page
            all_chunks = []
            for page_content in pages_content:
                chunks = self.create_smart_chunks(page_content)
                all_chunks.extend(chunks)
            
            if not all_chunks:
                return {phrase: 0 for phrase in criteria_phrases}
            
            # Get embeddings
            criteria_embeddings = self.get_embeddings(criteria_phrases)
            chunk_embeddings = self.get_embeddings(all_chunks)
            
            if criteria_embeddings.size == 0 or chunk_embeddings.size == 0:
                return {phrase: 0 for phrase in criteria_phrases}
            
            # Calculate similarities for each criteria phrase
            results = {}
            for i, phrase in enumerate(criteria_phrases):
                phrase_embedding = criteria_embeddings[i]
                max_similarity = 0
                
                for chunk_embedding in chunk_embeddings:
                    similarity = self.calculate_similarity(phrase_embedding, chunk_embedding)
                    max_similarity = max(max_similarity, similarity)
                
                results[phrase] = 1 if max_similarity >= self.similarity_threshold else 0
            
            return results
            
        except Exception as e:
            self.logger.error(f"Error in PDF analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}

    def read_criteria(self, excel_path: str) -> List[str]:
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]
            return processed_criteria
        except Exception as e:
            self.logger.error(f"Error reading criteria file: {e}")
            return []

    def analyze_multiple_pdfs(self, pdf_folder: str, excel_path: str, output_path: str):
        """Analyze multiple PDF files and save results"""
        try:
            # Read criteria
            criteria_phrases = self.read_criteria(excel_path)
            if not criteria_phrases:
                self.logger.error("No criteria found. Exiting...")
                return
            
            # Get list of PDF files
            pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
            if not pdf_files:
                self.logger.error("No PDF files found in the specified folder.")
                return
            
            # Process each PDF with progress bar
            all_results = []
            for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
                self.logger.info(f"Processing {pdf_file}...")
                pdf_path = os.path.join(pdf_folder, pdf_file)
                
                # Analyze PDF
                results = self.analyze_single_pdf(pdf_path, criteria_phrases)  ######################################################
                results['PDF File'] = pdf_file
                all_results.append(results)
            
            # Save results
            try:
                results_df = pd.DataFrame(all_results)
                results_df.to_excel(output_path, index=False)
                self.logger.info(f"Results saved to {output_path}")
            except Exception as e:
                self.logger.error(f"Error saving results: {e}")
                
        except Exception as e:
            self.logger.error(f"Error in multiple PDF analysis: {e}")

# Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
    analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing PDFs:   0%|                                                                           | 0/2 [00:00<?, ?it/s]INFO:__main__:Processing TH_milk.pdf...
ERROR:__main__:Error in chunk creation: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\VNC/nltk_data'
    - 'C:\\Users\\VNC\\Downloads\\Anaconda jpt\\nltk_data'
    - 'C:\\Users\\VNC\\Downloads\\Anaconda jpt\\share\\nltk_data'
    - 'C:\\Users\

In [3]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt')
    print("Punkt is installed correctly!")
except LookupError:
    print("Punkt is missing. Downloading now...")
    nltk.download('punkt')


List of fixture:
- Overlap 130 words, batch size optimal 32
- Read file by blocks
- 

In [39]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.74):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        full_text = []
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text = ""
                # Get text blocks with their positions
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                            text += ""
                # Process the text for this page
                processed_text = self.preprocess_text(text)
                full_text.append(processed_text)
        return ' '.join(full_text)
    
    def get_embeddings(self, texts, batch_size=32):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            processed_batch = [self.preprocess_text(text) for text in batch]
            
            encoded = self.pho_tokenizer(processed_batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract and preprocess text
        pdf_text = self.extract_text_from_pdf(pdf_path)
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into overlapping chunks
        words = pdf_text.split()
        chunk_size = 1000
        overlap_size = 130  
        chunks = []
        
        for i in range(0, len(words), chunk_size - overlap_size):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)
        
        if not chunks:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Get embeddings
        try:
            criteria_embeddings = self.get_embeddings(criteria_phrases)
            chunk_embeddings = self.get_embeddings(chunks)
            
            # Ensure proper shapes
            if len(criteria_embeddings.shape) == 1:
                criteria_embeddings = criteria_embeddings.reshape(1, -1)
            if len(chunk_embeddings.shape) == 1:
                chunk_embeddings = chunk_embeddings.reshape(1, -1)
            
            # Calculate similarities
            similarities = cosine_similarity(criteria_embeddings, chunk_embeddings)
            
            # Initialize and update results
            results = {}
            for i, phrase in enumerate(criteria_phrases):
                results[phrase] = 1 if np.max(similarities[i]) >= self.similarity_threshold else 0
            
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.74)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


In [6]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.8):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        full_text = []
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text = ""
                # Get text blocks with their positions
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                # Extract raw text from pdf
                full_text.append(text)
        return ' '.join(full_text)
    
    def get_embeddings(self, texts, batch_size=32):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into overlapping chunks
        
        sentences = self.chunk_into_sentences(pdf_text)        # a list contain X items aligning with X sentences in the pdf file
        if not sentences:
            return {phrase: 0 for phrase in criteria_phrases}

        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]        # Process one item, in the list of X items, per time
        
        # Get embeddings
        try:
            criteria_embeddings = self.get_embeddings(criteria_phrases)
            chunk_embeddings = self.get_embeddings(processed_text)
            
            # Ensure proper shapes
            if len(criteria_embeddings.shape) == 1:
                criteria_embeddings = criteria_embeddings.reshape(1, -1)
            if len(chunk_embeddings.shape) == 1:
                chunk_embeddings = chunk_embeddings.reshape(1, -1)
            
            # Calculate similarities
            similarities = cosine_similarity(criteria_embeddings, chunk_embeddings)
            
            # Initialize and update results
            results = {}
            for i, phrase in enumerate(criteria_phrases):
                results[phrase] = 1 if np.max(similarities[i]) >= self.similarity_threshold else 0
            
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.8)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing aaa-bao-cao-phat-trien-ben-vung-nam-2022-0.pdf...
Processing ACB ESG Report 2022 (Final).pdf...
Processing BC-Ben-vung-nam-2019.pdf...
Results saved to esg_analysis_results.xlsx


In [25]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.8):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        full_text = []
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text = ""
                # Get text blocks with their positions
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                # Extract raw text from pdf
                full_text.append(text)
        return ' '.join(full_text)
    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
            batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into overlapping chunks
        
        sentences = self.chunk_into_sentences(pdf_text)        # a list contain X items aligning with X sentences in the pdf file
        if not sentences:
            return {phrase: 0 for phrase in criteria_phrases}

        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]        # Process one item, in the list of X items, per time
        
        # Get embeddings
        try:
            criteria_embeddings = [self.get_embeddings(phrase) for phrase in criteria_phrases]
            
            results = {phrase: 0 for phrase in criteria_phrases}
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(criteria_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[criteria_phrases[i]] = 1
                    
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.8)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing aaa-bao-cao-phat-trien-ben-vung-nam-2022-0.pdf...
Processing ACB ESG Report 2022 (Final).pdf...
Processing BC-Ben-vung-nam-2019.pdf...
Results saved to esg_analysis_results.xlsx


In [9]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.67):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        full_text = []
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text = ""
                # Get text blocks with their positions
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                # Extract raw text from pdf
                full_text.append(text)
        return ' '.join(full_text)
    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
                # Use mean pooling instead of just [CLS] token
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            

            # L2 normalization for better cosine similarity
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
            batch_embeddings = batch_embeddings.numpy()
            
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Split into overlapping chunks
        
        sentences = self.chunk_into_sentences(pdf_text)        # a list contain X items aligning with X sentences in the pdf file
        if not sentences:
            return {phrase: 0 for phrase in criteria_phrases}

        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]        # Process one item, in the list of X items, per time

        # Get embeddings
        try:
            criteria_embeddings = [self.get_embeddings(phrase) for phrase in criteria_phrases]
            
            results = {phrase: 0 for phrase in criteria_phrases}
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(criteria_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[criteria_phrases[i]] = 1
                    
            return results
            
        except Exception as e:
            print(f"Error in analysis: {e}")
            return {phrase: 0 for phrase in criteria_phrases}

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.67)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words_test.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results_0.6.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing TH_milk.pdf...
Results saved to esg_analysis_results_0.6.xlsx


In [32]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.67):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file"""
        full_text = []
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text = ""
                # Get text blocks with their positions
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                # Extract raw text from pdf
                full_text.append(text)
        return ' '.join(full_text)
    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
                # Use mean pooling instead of just [CLS] token
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            

            # L2 normalization for better cosine similarity
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
            batch_embeddings = batch_embeddings.numpy()
            
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Initialize results dictionary
        results = {phrase: 0 for phrase in criteria_phrases}
        
        # First, perform exact matching
        pdf_text_lower = pdf_text.lower()
        for phrase in criteria_phrases:
            clean_phrase = self.clean_text(phrase).lower()
            if clean_phrase in pdf_text_lower:
                results[phrase] = 1
        
        # Get list of unmatched phrases for semantic analysis
        unmatched_phrases = [phrase for phrase in criteria_phrases if results[phrase] == 0]
        
        # If all phrases were matched exactly, return results
        if not unmatched_phrases:
            return results
            
        # Otherwise, proceed with semantic similarity analysis for unmatched phrases
        sentences = self.chunk_into_sentences(pdf_text)
        if not sentences:
            return results
    
        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]
        
        # Get embeddings only for unmatched phrases
        try:
            unmatched_embeddings = [self.get_embeddings(phrase) for phrase in unmatched_phrases]
            
            # Check semantic similarity only for unmatched phrases
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(unmatched_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[unmatched_phrases[i]] = 1
                
            return results
        
        except Exception as e:
            print(f"Error in semantic analysis: {e}")
            return results

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.67)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/words_test.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results_exactmatch_067.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing AME_Baocaothuongnien_2023-pages-2.pdf...
Processing TH_milk.pdf...
Results saved to esg_analysis_results_exactmatch_067.xlsx


In [1]:
!pip install pytesseract pdf2image opencv-python

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [2]:
!pip install torch transformers==4.50.3

  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached transformers-4.50.3-py3-none-any.whl (10.2 MB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [3]:
!pip install PyMuPDF

  Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.5-cp39-abi3-win_amd64.whl (16.6 MB)


## pip instALL CHO CODE 6

In [13]:
!pip install pyocr

In [6]:
# Set the path to tesseract executable
tesseract_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'Tesseract-OCR', 'tesseract.exe')
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# Verify Tesseract installation
try:
    print(f"Tesseract version: {pytesseract.get_tesseract_version()}")
except Exception as e:
    print(f"Error: {e}")
    print("Please make sure Tesseract is properly installed at:", tesseract_path)


Error: C:\Users\HP\Downloads\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.
Please make sure Tesseract is properly installed at: C:\Users\HP\Downloads\Tesseract-OCR\tesseract.exe


In [20]:
def test_ocr_with_vietnamese():
    # Create a test image with Vietnamese text
    img = np.zeros((200, 400, 3), dtype=np.uint8)
    cv2.putText(img, "Xin chào", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, "Tôi là máy tính", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Try OCR with Vietnamese
    try:
        text = pytesseract.image_to_string(img, lang='vie', config='--psm 6')
        print("OCR Test Result:", text)
    except Exception as e:
        print("OCR Test Failed:", e)

# Run the test
test_ocr_with_vietnamese()

OCR Test Result: Xin ch??o
T??i !?? m??y t??nh



In [22]:
def verify_tesseract_setup():
    # Check Tesseract version
    print("Tesseract version:", pytesseract.get_tesseract_version())
    
    # Check if Vietnamese language pack is installed
    tessdata_dir = os.path.join(os.path.expanduser('~'), 'Downloads', 'Tesseract-OCR', 'tessdata')
    vie_data_path = os.path.join(tessdata_dir, 'vie.traineddata')
    
    print("\nChecking Vietnamese language pack:")
    print("Path to vie.traineddata:", vie_data_path)
    print("File exists:", os.path.exists(vie_data_path))
    
    if os.path.exists(vie_data_path):
        print("File size:", os.path.getsize(vie_data_path), "bytes")
    
    # List all available languages
    print("\nAvailable languages:")
    try:
        langs = pytesseract.get_languages()
        print(langs)
    except Exception as e:
        print("Error getting languages:", e)

# Run the verification
verify_tesseract_setup()

Tesseract version: 5.5.0.20241111

Checking Vietnamese language pack:
Path to vie.traineddata: C:\Users\VNC\Downloads\Tesseract-OCR\tessdata\vie.traineddata
File exists: True
File size: 531275 bytes

Available languages:
['eng', 'osd', 'vie']


In [30]:
import cv2
import pytesseract
import os
import fitz  # PyMuPDF
import numpy as np

# Set your Tesseract path
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\VNC\Downloads\Tesseract-OCR\tesseract.exe'

def read_vietnamese_pdf(pdf_path):
    # Open the PDF
    with fitz.open(pdf_path) as doc:
        print(f"Processing PDF with {len(doc)} pages")
        
        for page_num, page in enumerate(doc, 1):
            print(f"\nProcessing page {page_num}")
            
            # Get images from the page
            image_list = page.get_images()
            print(f"Found {len(image_list)} images in page {page_num}")
            
            for img_index, img in enumerate(image_list):
                try:
                    # Get image data
                    xref = img[0]
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    
                    # Convert to numpy array
                    nparr = np.frombuffer(image_bytes, np.uint8)
                    img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                    
                    # Preprocess the image
                    # Convert to grayscale
                    gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
                    
                    # Apply thresholding
                    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                    
                    # Save preprocessed image for debugging
                    debug_path = f'preprocessed_page_{page_num}_img_{img_index}.png'
                    cv2.imwrite(debug_path, thresh)
                    
                    # Try different PSM modes
                    psm_modes = ['--psm 6']
                    
                    print(f"\nResults for image {img_index} on page {page_num}:")
                    for psm in psm_modes:
                        text = pytesseract.image_to_string(
                            thresh,
                            lang='vie',
                            config=psm
                        )
                        if text.strip():  # Only print if text was found
                            print(f"\nUsing {psm}:")
                            print(text.strip())
                            
                except Exception as e:
                    print(f"Error processing image {img_index} on page {page_num}: {e}")
                    continue

# Example usage
if __name__ == "__main__":
    # Replace with your PDF path
    pdf_path = "Downloads/AME_Baocaothuongnien_2023-pages-2.pdf"  # Update this path
    read_vietnamese_pdf(pdf_path)

Processing PDF with 3 pages

Processing page 1
Found 1 images in page 1

Results for image 0 on page 1:

Using --psm 6:
CÔNG TY CỎ PHẢN ALPHANAM E&C

Tâng 3, sô 108 Nguyên Trãi, Phường Thượng Đình, quận Thanh Xuân, Hà Nội, Việt Nam
Các cuộc họp của Hội đồng quản trị trong năm, Tổng Giám đốc và các thành viên được mời
tham dự và báo cáo, đông thời thông báo đây đủ các thông tin vê nội dung cuộc họp đề đảm
báo Tổng Giám đốc thực hiện nghiêm túc các nhiệm vụ và quyền hạn theo đúng quy định.
Trong năm, Hội đồng quản trị không ghi nhận bất thường trong các hoạt động quản lý và
điều hành của Tổng Giám đốc và các cán bộ quản lý.

3. Các kế hoạch, định hướng của Hội đồng quản trị :

- __ Phát triển ôn định và bền vững các lĩnh vực cốt lõi, truyền thông.

- _ Tim hiểu, tận dụng các cơ hội thị trường để mở rộng đầu tư vào các dự án chiến lược, có

tiềm năng cao về số lượng và chất lượng cho Công ty đầu tư.

- _ Chú trọng công tác phát triển nguồn nhân lực có chất lượng cao, phát triển đội ngũ qu

In [1]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
from pdf2image import convert_from_path
import cv2
import numpy as np
from collections import defaultdict

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file, including text from images"""
        full_text = []
        
        with fitz.open(pdf_path) as doc:
            for page in doc:
                # Extract regular PDF text
                text = ""
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                full_text.append(text)
                
                # Extract text from images in the page
                image_list = page.get_images()
                for img_index, img in enumerate(image_list):
                    try:
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        
                        # Convert image bytes to OpenCV format
                        nparr = np.frombuffer(image_bytes, np.uint8)
                        img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        
                        # === Preprocessing pipeline ===
    
                        # 1. Grayscale conversion
                        gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
                        # 2. Brightness normalization (CLAHE)
                        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                        gray = clahe.apply(gray)
    
                        # 3. Resize to improve Vietnamese diacritic detection
                        height = gray.shape[0]
                        if height < 1000:
                            scale = 300 / 72  # Simulate ~300 DPI if lower res
                            gray = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    
                        # 4. Thresholding with Otsu
                        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
                        # 5. Deskewing (if needed)
                        coords = np.column_stack(np.where(thresh > 0))
                        angle = cv2.minAreaRect(coords)[-1]
                        if angle < -45:
                            angle = 90 + angle
                        elif angle > 45:
                            angle = angle - 90
                        (h, w) = thresh.shape
                        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                        deskewed = cv2.warpAffine(thresh, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    
                        # 6. Light morphological closing to connect broken strokes
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
                        processed = cv2.morphologyEx(deskewed, cv2.MORPH_CLOSE, kernel)
    
                        # === Tesseract OCR with layout preservation ===
                        d = pytesseract.image_to_data(
                            processed,
                            lang='vie',
                            config='--oem 3 --psm 6',
                            output_type=pytesseract.Output.DICT
                        )
    
                        # Group words into structured text (block → line → word)
                        blocks = defaultdict(lambda: defaultdict(list))
                        for i in range(len(d['text'])):
                            if int(d['conf'][i]) > 0 and d['text'][i].strip():
                                block_id = d['block_num'][i]
                                line_id = d['line_num'][i]
                                word_info = {
                                    'text': d['text'][i],
                                    'left': d['left'][i],
                                    'top': d['top'][i]
                                }
                                blocks[block_id][line_id].append(word_info)
    
                        # Sort blocks and lines spatially
                        sorted_blocks = sorted(blocks.items(), key=lambda b: min([w['top'] for line in b[1].values() for w in line]))
                        structured_text = ""
    
                        for block_id, lines in sorted_blocks:
                            sorted_lines = sorted(lines.items(), key=lambda l: min(w['top'] for w in l[1]))
                            for line_id, words in sorted_lines:
                                sorted_words = sorted(words, key=lambda w: w['left'])
                                line_text = " ".join([w['text'] for w in sorted_words])
                                structured_text += line_text.strip() + "\n"
    
                        # Append image-derived text
                        full_text.append(structured_text)
    
                    except Exception as e:
                        print(f"Error processing image {img_index} on page {page.number}: {e}")
                        continue
    
        return "\n".join(full_text)

    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
                # Use mean pooling instead of just [CLS] token
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            

            # L2 normalization for better cosine similarity
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
            batch_embeddings = batch_embeddings.numpy()
            
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Initialize results dictionary
        results = {phrase: 0 for phrase in criteria_phrases}
        
        # First, perform exact matching
        pdf_text_lower = pdf_text.lower()
        for phrase in criteria_phrases:
            clean_phrase = self.clean_text(phrase).lower()
            if clean_phrase in pdf_text_lower:
                results[phrase] = 1
        
        # Get list of unmatched phrases for semantic analysis
        unmatched_phrases = [phrase for phrase in criteria_phrases if results[phrase] == 0]
        
        # If all phrases were matched exactly, return results
        if not unmatched_phrases:
            return results
            
        # Otherwise, proceed with semantic similarity analysis for unmatched phrases
        sentences = self.chunk_into_sentences(pdf_text)
        if not sentences:
            return results
    
        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]
        
        # Get embeddings only for unmatched phrases
        try:
            unmatched_embeddings = [self.get_embeddings(phrase) for phrase in unmatched_phrases]
            
            # Check semantic similarity only for unmatched phrases
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(unmatched_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[unmatched_phrases[i]] = 1
                
            return results
        
        except Exception as e:
            print(f"Error in semantic analysis: {e}")
            return results

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return
        
        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
        
        # Process each PDF
        all_results = []
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
        # Save results
        try:
            results_df = pd.DataFrame(all_results)
            results_df.to_excel(output_path, index=False)
            print(f"Results saved to {output_path}")
        except Exception as e:
            print(f"Error saving results: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/esg_words.xlsx"  # Replace with your Excel file path
    output_path = "esg_analysis_results_exactmatch_imageread.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing 2012 - ACC.pdf...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
from pdf2image import convert_from_path
import cv2
import numpy as np
from collections import defaultdict

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords
            "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", 
            "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", 
            "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", 
            "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", 
            "qua", "ra", "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", 
            "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF file, including text from images"""
        full_text = []
        
        with fitz.open(pdf_path) as doc:
            for page in doc:
                # Extract regular PDF text
                text = ""
                blocks = page.get_text("dict")["blocks"]
                for block in blocks:
                    if "lines" in block:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                text += span["text"] + " "
                full_text.append(text)
                
                # Extract text from images in the page
                image_list = page.get_images()
                for img_index, img in enumerate(image_list):
                    try:
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_bytes = base_image["image"]
                        
                        # Convert image bytes to OpenCV format
                        nparr = np.frombuffer(image_bytes, np.uint8)
                        img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        
                        # === Preprocessing pipeline ===
    
                        # 1. Grayscale conversion
                        gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
                        # 2. Brightness normalization (CLAHE)
                        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                        gray = clahe.apply(gray)
    
                        # 3. Resize to improve Vietnamese diacritic detection
                        height = gray.shape[0]
                        if height < 1000:
                            scale = 300 / 72  # Simulate ~300 DPI if lower res
                            gray = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    
                        # 4. Thresholding with Otsu
                        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
                        # 5. Deskewing (if needed)
                        coords = np.column_stack(np.where(thresh > 0))
                        angle = cv2.minAreaRect(coords)[-1]
                        if angle < -45:
                            angle = 90 + angle
                        elif angle > 45:
                            angle = angle - 90
                        (h, w) = thresh.shape
                        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                        deskewed = cv2.warpAffine(thresh, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    
                        # 6. Light morphological closing to connect broken strokes
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
                        processed = cv2.morphologyEx(deskewed, cv2.MORPH_CLOSE, kernel)
    
                        # === Tesseract OCR with layout preservation ===
                        d = pytesseract.image_to_data(
                            processed,
                            lang='vie',
                            config='--oem 3 --psm 6',
                            output_type=pytesseract.Output.DICT
                        )
    
                        # Group words into structured text (block → line → word)
                        blocks = defaultdict(lambda: defaultdict(list))
                        for i in range(len(d['text'])):
                            if int(d['conf'][i]) > 0 and d['text'][i].strip():
                                block_id = d['block_num'][i]
                                line_id = d['line_num'][i]
                                word_info = {
                                    'text': d['text'][i],
                                    'left': d['left'][i],
                                    'top': d['top'][i]
                                }
                                blocks[block_id][line_id].append(word_info)
    
                        # Sort blocks and lines spatially
                        sorted_blocks = sorted(blocks.items(), key=lambda b: min([w['top'] for line in b[1].values() for w in line]))
                        structured_text = ""
    
                        for block_id, lines in sorted_blocks:
                            sorted_lines = sorted(lines.items(), key=lambda l: min(w['top'] for w in l[1]))
                            for line_id, words in sorted_lines:
                                sorted_words = sorted(words, key=lambda w: w['left'])
                                line_text = " ".join([w['text'] for w in sorted_words])
                                structured_text += line_text.strip() + "\n"
    
                        # Append image-derived text
                        full_text.append(structured_text)
    
                    except Exception as e:
                        print(f"Error processing image {img_index} on page {page.number}: {e}")
                        continue
    
        return "\n".join(full_text)

    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
                # Use mean pooling instead of just [CLS] token
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            

            # L2 normalization for better cosine similarity
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
            batch_embeddings = batch_embeddings.numpy()
            
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: 0 for phrase in criteria_phrases}
        
        # Initialize results dictionary
        results = {phrase: 0 for phrase in criteria_phrases}
        
        # First, perform exact matching
        pdf_text_lower = pdf_text.lower()
        for phrase in criteria_phrases:
            clean_phrase = self.clean_text(phrase).lower()
            if clean_phrase in pdf_text_lower:
                results[phrase] = 1
        
        # Get list of unmatched phrases for semantic analysis
        unmatched_phrases = [phrase for phrase in criteria_phrases if results[phrase] == 0]
        
        # If all phrases were matched exactly, return results
        if not unmatched_phrases:
            return results
            
        # Otherwise, proceed with semantic similarity analysis for unmatched phrases
        sentences = self.chunk_into_sentences(pdf_text)
        if not sentences:
            return results
    
        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]
        
        # Get embeddings only for unmatched phrases
        try:
            unmatched_embeddings = [self.get_embeddings(phrase) for phrase in unmatched_phrases]
            
            # Check semantic similarity only for unmatched phrases
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(unmatched_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[unmatched_phrases[i]] = 1
                
            return results
        
        except Exception as e:
            print(f"Error in semantic analysis: {e}")
            return results

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return

        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
            
        all_results = []   
        for idx, pdf_file in enumerate(pdf_files, 1):  # Start index at 1
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
        
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
            # Save after every 3 files OR at the end of the list
            if idx % 2 == 0 or idx == len(pdf_files): 
                try:
                    results_df = pd.DataFrame(all_results)
                    results_df.to_excel(output_path, index=False)
                    print(f"Intermediate results saved to {output_path} after processing {idx} PDFs.")
                except Exception as e:
                    print(f"Error saving results after {idx} PDFs: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/esg_words.xlsx"  # Replace with your Excel file path
    output_path = "esg_pdf_reading.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing 2011 - HU1.pdf...
Processing 2011 - HUT.pdf...
Intermediate results saved to esg_pdf_reading.xlsx after processing 2 PDFs.
Processing 2011 - HVX.pdf...
Processing 2011 - IDI.pdf...
Intermediate results saved to esg_pdf_reading.xlsx after processing 4 PDFs.
Processing 2011 - IMP.pdf...
Processing 2012 - HAS.pdf...


In [ ]:
!pip install PyMuPDF, transformers==4.50.3, torch==2.6.0, pytesseract, pdf2image, opencv-python

# Final V3

In [1]:
import pandas as pd
import fitz  # PyMuPDF
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import os
import re
from vws import RDRSegmenter, Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
from pdf2image import convert_from_path
import cv2
import numpy as np
from collections import defaultdict

class ESGAnalyzer:
    def __init__(self, similarity_threshold=0.65):
        # Initialize PhoBERT
        self.pho_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2") 
        self.model = AutoModel.from_pretrained("vinai/phobert-base-v2")

        # NEW: Initialize Vietnamese word segmenter
        self.rdrsegment = RDRSegmenter.RDRSegmenter()  # Changed variable name
        self.token = Tokenizer.Tokenizer()
        
        # Vietnamese stopwords - expanded for ESG context
        self.stopwords = set([
            # Common Vietnamese stopwords           
        "bị", "bởi", "cả", "các", "có", "có_thể", "có_lẽ", "của",
        "cùng","cùng_với", "cũng", "đã", "đang", "đây", "để", "đều", "do", "đó",
        "khi", "là", "lại", "mà", "nên", "nếu", "những",
        "phải", "rất", "rồi", "sau", "sẽ", "thì", "từ", "và"
        ])
        
        self.similarity_threshold = similarity_threshold


    
    def segment_words(self, text):
        """Segment Vietnamese text using vws"""
        # Use segmentRawSentences to process the text
        segmented_text = self.rdrsegment.segmentRawSentences(self.token, text)
        return segmented_text
    
    def clean_text(self, text):
        """Clean and normalize Vietnamese text for ESG content"""
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters but keep Vietnamese characters and numbers
        text = re.sub(r'[^\w\s\dàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]', ' ', text)
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def remove_stopwords(self, text):
        """Remove Vietnamese stopwords"""
        words = text.split()
        filtered_words = [word for word in words if word not in self.stopwords]
        return ' '.join(filtered_words)
    
    def preprocess_text(self, text):
        """Complete text preprocessing pipeline"""
        # Clean text (unchanged)
        text = self.clean_text(text)
        
        # Remove stopwords (unchanged)
        text = self.remove_stopwords(text)
        
        # Use vws for word segmentation instead of simple split
        segmented_text = self.segment_words(text)
        
        # Return segment words
        return segmented_text

    def extract_text_from_pdf(self, pdf_path):
        """Extract and preprocess text from PDF, handling both vector and image-only pages"""

        full_text = []
        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    # === 1. Extract Vector Text ===
                    text = ""
                    blocks = page.get_text("dict")["blocks"]
                    for block in blocks:
                        if "lines" in block:
                            for line in block["lines"]:
                                for span in line["spans"]:
                                    text += span["text"] + " "
                    full_text.append(text.strip())
        
                    # === 2. OCR Page as Image (Visual Handling) ===
                    try:
                        zoom = 2.0  # Higher DPI (144 DPI)
                        mat = fitz.Matrix(zoom, zoom)
                        pix = page.get_pixmap(matrix=mat)
        
                        img_np = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, pix.n))
                        if pix.n == 4:
                            img_np = cv2.cvtColor(img_np, cv2.COLOR_RGBA2BGR)
                        elif pix.n == 1:
                            img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
        
                        # Quick precheck (low-cost confidence screening)
                        quick_ocr = pytesseract.image_to_data(
                            img_np, lang='vie', config='--oem 3 --psm 6', output_type=pytesseract.Output.DICT
                        )
                        quick_valid_chars = sum(len(word.strip()) for word, conf in zip(quick_ocr['text'], quick_ocr['conf']) if word.strip() and int(conf) > 60)
        
                        if quick_valid_chars < 6:
                            continue
        
                        # === Preprocessing for final OCR ===
                        gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
                        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                        gray = clahe.apply(gray)
        
                        # Upscale if too small
                        if gray.shape[0] < 1000:
                            scale = 300 / 72
                            gray = cv2.resize(gray, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        
                        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
                        coords = np.column_stack(np.where(thresh > 0))
                        angle = cv2.minAreaRect(coords)[-1]
                        if angle < -45:
                            angle += 90
                        elif angle > 45:
                            angle -= 90
                        (h, w) = thresh.shape
                        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                        deskewed = cv2.warpAffine(thresh, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        
                        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
                        processed = cv2.morphologyEx(deskewed, cv2.MORPH_CLOSE, kernel)
        
                        d = pytesseract.image_to_data(
                            processed, lang='vie', config='--oem 3 --psm 6', output_type=pytesseract.Output.DICT
                        )
        
                        # === Filter out meaningless pages ===
                        valid_word_count = 0
                        valid_char_count = 0
                        for i in range(len(d['text'])):
                            word = d['text'][i].strip()
                            conf = int(d['conf'][i])
                            if conf >= 70 and len(word) >= 3:
                                valid_word_count += 1
                                valid_char_count += len(word)
        
                        if valid_word_count < 3 or valid_char_count < 17:
                            continue
        
                        # === Structure OCR Text ===
                        blocks = defaultdict(lambda: defaultdict(list))
                        for i in range(len(d['text'])):
                            if int(d['conf'][i]) > 0 and d['text'][i].strip():
                                block_id = d['block_num'][i]
                                line_id = d['line_num'][i]
                                word_info = {
                                    'text': d['text'][i],
                                    'left': d['left'][i],
                                    'top': d['top'][i]
                                }
                                blocks[block_id][line_id].append(word_info)
        
                        sorted_blocks = sorted(blocks.items(), key=lambda b: min([w['top'] for line in b[1].values() for w in line]))
                        structured_text = ""
                        for block_id, lines in sorted_blocks:
                            sorted_lines = sorted(lines.items(), key=lambda l: min(w['top'] for w in l[1]))
                            for line_id, words in sorted_lines:
                                sorted_words = sorted(words, key=lambda w: w['left'])
                                line_text = " ".join([w['text'] for w in sorted_words])
                                structured_text += line_text.strip() + "\n"
        
                        full_text.append(structured_text)
        
                    except Exception as e:
                        print(f"❌ Error rendering or OCRing page {page.number}: {e}")
                        continue
        
            return "\n".join(full_text)
        except Exception as e:
            print(f"Skipped empty file: {pdf_path}")
            
    
    def get_embeddings(self, texts, batch_size=1):
        """Get embeddings in batches"""
        if isinstance(texts, str):
            texts = [texts]
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            
            
            encoded = self.pho_tokenizer(batch, 
                                   padding=True, 
                                   truncation=True,
                                   return_tensors="pt",
                                   max_length=256)
            
            with torch.no_grad():
                outputs = self.model(**encoded)
            
                # Use mean pooling instead of just [CLS] token
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            

            # L2 normalization for better cosine similarity
            batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
            batch_embeddings = batch_embeddings.numpy()
            
            all_embeddings.append(batch_embeddings)
        
        return np.vstack(all_embeddings)
    
    def read_criteria(self, excel_path):
        """Read and preprocess ESG criteria from Excel file"""
        try:
            df = pd.read_excel(excel_path)
            # Ensure we're reading just one column
            if 'criteria' in df.columns:
                criteria = df['criteria'].tolist()           # A list of criteria is collected
            else:
                # If 'criteria' column doesn't exist, take the first column
                criteria = df.iloc[:, 0].tolist()
            
            processed_criteria = [self.preprocess_text(str(c)) for c in criteria]      # Process 1 item per time in a list of criteria
            return processed_criteria
        except Exception as e:
            print(f"Error reading criteria file: {e}")
            return []

    def chunk_into_sentences(self, text):
        """Chunk the text into sentences based on delimiters (. ? ! ; ...)."""
        sentence_endings = re.compile(r'([.!?;]{1,3})')  # Regex to match sentence-ending delimiters
        sentences = re.split(sentence_endings, text)
        sentences = [sentences[i].strip() + (sentences[i + 1] if i + 1 < len(sentences) else '')
                     for i in range(0, len(sentences), 2)]  # Only take sentences (even index)
        return [s.strip() for s in sentences if s.strip()]
        
    def analyze_single_pdf(self, pdf_path, criteria_phrases):
        """Analyze a single PDF file against criteria phrases"""
        # Extract raw pdf text
        pdf_text = self.extract_text_from_pdf(pdf_path)      # full pdf file, raw and original
        if not pdf_text:
            return {phrase: "x" for phrase in criteria_phrases}

                 
        # Initialize results dictionary
        results = {phrase: 0 for phrase in criteria_phrases}
        
        # First, perform exact matching
        pdf_text_lower = pdf_text.lower()
        for phrase in criteria_phrases:
            clean_phrase = self.clean_text(phrase).lower()
            if clean_phrase in pdf_text_lower:
                results[phrase] = 1
        
        # Get list of unmatched phrases for semantic analysis
        unmatched_phrases = [phrase for phrase in criteria_phrases if results[phrase] == 0]
        
        # If all phrases were matched exactly, return results
        if not unmatched_phrases:
            return results
            
        # Otherwise, proceed with semantic similarity analysis for unmatched phrases
        sentences = self.chunk_into_sentences(pdf_text)
        if not sentences:
            return results
    
        # Preprocess_text so it is cleaned
        processed_text = [self.preprocess_text(senten) for senten in sentences]
        
        # Get embeddings only for unmatched phrases
        try:
            unmatched_embeddings = [self.get_embeddings(phrase) for phrase in unmatched_phrases]
            
            # Check semantic similarity only for unmatched phrases
            for sentence in processed_text:
                sentence_embedding = self.get_embeddings(sentence)
                if len(sentence_embedding.shape) == 1:
                    sentence_embedding = sentence_embedding.reshape(1, -1)
                for i, criterion_embedding in enumerate(unmatched_embeddings):
                    similarity = cosine_similarity(sentence_embedding, criterion_embedding)[0][0]
                    if similarity >= self.similarity_threshold:
                        results[unmatched_phrases[i]] = 1
                
            return results
        
        except Exception as e:
            print(f"Error in semantic analysis: {e}")
            return results

    
    def analyze_multiple_pdfs(self, pdf_folder, excel_path, output_path):
        """Analyze multiple PDF files and save results"""
        # Read criteria
        criteria_phrases = self.read_criteria(excel_path)
        if not criteria_phrases:
            print("No criteria found. Exiting...")
            return

        # Get list of PDF files
        pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the specified folder.")
            return
            
        all_results = []   
        for idx, pdf_file in enumerate(pdf_files, 1):  # Start index at 1
            print(f"Processing {pdf_file}...")
            pdf_path = os.path.join(pdf_folder, pdf_file)
        
            # Analyze PDF
            results = self.analyze_single_pdf(pdf_path, criteria_phrases)
            results['PDF File'] = pdf_file
            all_results.append(results)
        
            # Save after every 2 files OR at the end of the list
            if idx % 2 == 0 or idx == len(pdf_files): 
                try:
                    results_df = pd.DataFrame(all_results)
                    results_df.to_excel(output_path, index=False)
                    print(f"Intermediate results saved to {output_path} after processing {idx} PDFs.")
                except Exception as e:
                    print(f"Error saving results after {idx} PDFs: {e}")
            
#Example usage
if __name__ == "__main__":
    analyzer = ESGAnalyzer(similarity_threshold=0.65)
    
    # Set paths
    pdf_folder = "Desktop/ESG pdf"  # Replace with your PDF folder path
    excel_path = "Desktop/esg_words.xlsx"  # Replace with your Excel file path
    output_path = "esg_pdf_reading.xlsx"
    
    # Run analysis
analyzer.analyze_multiple_pdfs(pdf_folder, excel_path, output_path)



KeyboardInterrupt

